In [1]:
# 전날 종가가 고가 또는 저가와 가까울 때의 다음날 시가 사이의 수익률 비교
import pandas as pd
from antQuant.utils import DataManager

In [2]:
stock = DataManager()

In [3]:
data = stock.read_data('AH_DB', 'adjdailyohlcv',['date', 'code', 'close', 'open','high','low'], conditions = {'date': "BETWEEN DATE ' 2016-01-01' AND DATE '2018-05-01'"})

In [4]:
data.date = pd.to_datetime(data.date)
data.head()

,date,code,close,open,high,low
0,2016-01-04,001040,244000.0,247500.0,249000.0,244000.0
1,2016-01-04,001045,123500.0,128000.0,130000.0,123500.0
2,2016-01-04,001070,12720.0,12740.0,13100.0,12680.0
3,2016-01-04,002450,3430.0,3520.0,3520.0,3420.0
4,2016-01-04,002600,160000.0,163000.0,170000.0,156500.0


In [5]:
data.set_index(['date', 'code'], inplace=True)
data.head()

close      open      high       low
date       code                                          
2016-01-04 001040  244000.0  247500.0  249000.0  244000.0
           001045  123500.0  128000.0  130000.0  123500.0
           001070   12720.0   12740.0   13100.0   12680.0
           002450    3430.0    3520.0    3520.0    3420.0
           002600  160000.0  163000.0  170000.0  156500.0

In [6]:
import numpy as np

In [7]:
# 다음날 시가를 당일 종가로 맞추기
data['opend1'] = data['open'].groupby(level=1).shift(-1)
data.head()

close      open      high       low    opend1
date       code                                                    
2016-01-04 001040  244000.0  247500.0  249000.0  244000.0  241500.0
           001045  123500.0  128000.0  130000.0  123500.0  124500.0
           001070   12720.0   12740.0   13100.0   12680.0   12720.0
           002450    3430.0    3520.0    3520.0    3420.0    3370.0
           002600  160000.0  163000.0  170000.0  156500.0  160000.0

In [8]:
data['returns'] = np.log(data['opend1']/data['close'])
data = data[(data['returns']!=np.inf)&(data['returns']!=-np.inf)]
data.head()

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: RuntimeWarning: divide by zero encountered in log
  if __name__ == '__main__':


close      open      high       low    opend1   returns
date       code                                                              
2016-01-04 001040  244000.0  247500.0  249000.0  244000.0  241500.0 -0.010299
           001045  123500.0  128000.0  130000.0  123500.0  124500.0  0.008065
           001070   12720.0   12740.0   13100.0   12680.0   12720.0  0.000000
           002450    3430.0    3520.0    3520.0    3420.0    3370.0 -0.017648
           002600  160000.0  163000.0  170000.0  156500.0  160000.0  0.000000

In [16]:
# 전날 종가가 고가와 가까울 때의 다음날 시가 사이의 수익률 비교
print('sum of returns: {0}'.format(data[(data['high'] - data['close']) < (data['close'] - data['low'])]['returns'].dropna().sum()))
print(data[(data['high'] - data['close']) < (data['close'] - data['low'])]['returns'].dropna().describe())

sum of returns: 92.57035774505937
count    637029.000000
mean          0.000145
std           0.023536
min          -2.353969
25%          -0.003824
50%           0.000000
75%           0.004812
max           4.598801
Name: returns, dtype: float64


In [17]:
# 전날 종가가 저가와 가까울 때의 다음날 시가 사이의 수익률 비교
print('sum of returns: {0}'.format(data[(data['high'] - data['close']) > (data['close'] - data['low'])]['returns'].dropna().sum()))
print(data[(data['high'] - data['close']) > (data['close'] - data['low'])]['returns'].dropna().describe())

sum of returns: 1143.9859270568886
count    656417.000000
mean          0.001743
std           0.018752
min          -1.427116
25%          -0.003017
50%           0.000000
75%           0.006335
max           2.261763
Name: returns, dtype: float64
